In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix,roc_curve,auc,accuracy_score

pd.options.display.html.table_schema=True
pd.options.display.max_rows=None
pd.options.display.max_columns=None

In [11]:
data=pd.read_excel('111.xlsx',sheet_name='Лист2')
X=np.array(data.drop('Событие', axis=1))
y=np.array(data['Событие'])

array([0, 0, 0, 0, 1, 1, 1, 1, 4, 0, 0, 2, 1, 0, 0, 0, 0, 0, 3, 0, 0, 0,
       0, 2, 4, 4, 0, 2, 1], dtype=int64)

In [13]:
rand_forest=RandomForestClassifier(random_state=0, n_estimators=840, max_depth=5).fit(X, y)

In [22]:
[print(data.columns[index],x) for index,x in enumerate(rand_forest.feature_importances_) if x>0.05]

Покраснение 0.06948070012182993
неприятные ощущения в глазу – присутствие – имеется 0.07206658763845382
конъюнктива ВиН век – изменения – отек 0.06108336589827186
конъюнктива ВиН век -  характер отделяемого 0.05970203242649546


[None, None, None, None]

In [27]:
print(rand_forest.score(X,y))
y_pred=rand_forest.predict(X)
pd.crosstab(y,y_pred)

0.9655172413793104


col_0,0,1,2,4
row_0,,,,
0,16,0,0,0
1,0,6,0,0
2,0,0,3,0
3,0,1,0,0
4,0,0,0,3


In [30]:
New_X=data[['Покраснение','неприятные ощущения в глазу – присутствие – имеется','конъюнктива ВиН век – изменения – отек','конъюнктива ВиН век -  характер отделяемого']]
rand_forest1=RandomForestClassifier(random_state=0, n_estimators=840, max_depth=5).fit(New_X, y)
print(rand_forest1.score(New_X,y))

0.8275862068965517
